In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def show(img, size=75):
    plt.figure(figsize=(size, size))

    if len(img.shape) == 2:  # if grayscale, set cmap
        plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    else:  # if 3 channels, change from BGR to RGB
        plt.imshow(img[..., ::-1])

    plt.show()

def get_first_frame(videofile):
    vidcap = cv2.VideoCapture(videofile)
    success, image = vidcap.read()
    if success:
        cv2.imwrite("book_video_first_frame.jpg", image)  # save frame as JPEG file


def get_keypoints(img):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(img,None)
    img_keypoints=cv2.drawKeypoints(img,kp,img)
    return kp, des


def get_correspondences(img1, img2):
    kp1, des1 = get_keypoints(img1)
    kp2, des2 = get_keypoints(img2)

    # BFMatcher with default params
    bf = cv2.BFMatcher()

    # des1 = queryDescriptor, des2 = trainDescriptor
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    final_matches = []
    for m, n in matches:
        if m.distance < 0.235 * n.distance:
            good.append([m])
            final_matches.append([kp1[m.queryIdx].pt, kp2[m.trainIdx].pt])

    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    # show(img3)
    print(f"found {len(good)} matches")

    return final_matches, img3


def construct_homography_matrix(matches):
    A = [];
    for match in matches:
        xi = match[0][0]
        yi = match[0][1]

        xj = match[1][0]
        yj = match[1][1]

        A.append([xi, yi, 1, 0, 0, 0, -1 * xj * xi, -1 * xj * yi, -1 * xj])
        A.append([0, 0, 0, xi, yi, 1, -1 * yj * xi, -1 * yj * yi, -1 * yj])

    # print(A)

    u, s, vh = np.linalg.svd(A, full_matrices=True)
    h = vh[-1]
    return np.array(h).reshape((3, 3))


In [18]:
get_first_frame('book.mov')
img1 = cv2.imread('cv_cover.jpg')
img2 = cv2.imread('book_video_first_frame.jpg')

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

final_matches, img3 = get_correspondences(img1, img2)
cv2.imshow('SIFT', img3)
cv2.waitKey(0)

H = construct_homography_matrix(final_matches) # The homography matrix
# print(H)

img_out = cv2.warpPerspective(img1, H, (img2.shape[1], img2.shape[0]))

cv2.imshow('Warp Perspective', img_out)
cv2.waitKey(0)

found 34 matches


-1